In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import os

In [10]:
# Define dataset paths
train_dir = r"C:\Users\DELL\Desktop\D4\train"
valid_dir = r"C:\Users\DELL\Desktop\D4\valid"
test_dir = r"C:\Users\DELL\Desktop\D4\test"

# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50

In [11]:
# Data augmentation and preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = data_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_gen = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 1879 images belonging to 3 classes.
Found 198 images belonging to 3 classes.
Found 183 images belonging to 3 classes.


In [12]:
# Handle class imbalance
from tensorflow.keras.applications import ResNet50
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array(list(range(len(train_gen.class_indices)))),  # Convert to NumPy array
    y=train_gen.classes
)
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Load pre-trained model (DenseNet121)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(len(train_gen.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)



In [13]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

In [14]:
# Train the model
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)


C:\Users\DELL\Desktop\PFETesting\paper1\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.3717 - loss: 1.4675

59/59 ━━━━━━━━━━━━━━━━━━━━ 323s 5s/step - accuracy: 0.3729 - loss: 1.4646 - val_accuracy: 0.7828 - val_loss: 0.5772
Epoch 2/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6661 - loss: 0.8090

59/59 ━━━━━━━━━━━━━━━━━━━━ 292s 5s/step - accuracy: 0.6660 - loss: 0.8093 - val_accuracy: 0.8737 - val_loss: 0.3829
Epoch 3/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6726 - loss: 0.7207

59/59 ━━━━━━━━━━━━━━━━━━━━ 319s 5s/step - accuracy: 0.6730 - loss: 0.7201 - val_accuracy: 0.8788 - val_loss: 0.3770
Epoch 4/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7245 - loss: 0.6321

...

59/59 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.7948 - loss: 0.5104 - val_accuracy: 0.9040 - val_loss: 0.2369
Epoch 14/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.8138 - loss: 0.4397 - val_accuracy: 0.8889 - val_loss: 0.2607
Epoch 15/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8296 - loss: 0.4752

59/59 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.8296 - loss: 0.4745 - val_accuracy: 0.9040 - val_loss: 0.2351
Epoch 16/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8419 - loss: 0.4233

59/59 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.8415 - loss: 0.4237 - val_accuracy: 0.9040 - val_loss: 0.2306
Epoch 17/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8207 - loss: 0.4241

59/59 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.8208 - loss: 0.4241 - val_accuracy: 0.9091 - val_loss: 0.2146
Epoch 18/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.8209 - loss: 0.4046 - val_accuracy: 0.9091 - val_loss: 0.2279
Epoch 19/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8216 - loss: 0.4507

59/59 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.8217 - loss: 0.4504 - val_accuracy: 0.9141 - val_loss: 0.2087
Epoch 20/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 176s 3s/step - accuracy: 0.8132 - loss: 0.4569 - val_accuracy: 0.9091 - val_loss: 0.2137
Epoch 21/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 165s 3s/step - accuracy: 0.8466 - loss: 0.3893 - val_accuracy: 0.9091 - val_loss: 0.2175
Epoch 22/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.8278 - loss: 0.3723 - val_accuracy: 0.8990 - val_loss: 0.2194
Epoch 23/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.8519 - loss: 0.3483 - val_accuracy: 0.9091 - val_loss: 0.2390
Epoch 24/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.8393 - loss: 0.3906 - val_accuracy: 0.9040 - val_loss: 0.2353


In [15]:
# Evaluate the model
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Unfreeze the base model for fine-tuning
base_model.trainable = True

# Recompile with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.9216 - loss: 0.2520
Test Accuracy: 87.98%


In [ ]:
# Fine-tune the model
history_fine_tune = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks
)

# Save the final model
model.save('wheat_disease_classifier.h5')

Epoch 1/10
